In [33]:
import pyterrier as pt
import json
import numpy as np
from scipy.spatial.distance import jensenshannon
import matplotlib.pyplot as plt
import ast
import pandas as pd
import ir_datasets
if not pt.started():
    pt.init(tqdm="notebook")

# 1. Define evaluation metrics

In [5]:
def dcg(scores, k):
    return np.sum([(rel / np.log2(idx + 2)) for idx, rel in enumerate(scores[:k], start=1)])

def calculate_ndcg(ranked_list, qrel_dict, qid, n):
    """
    Calculate the nDCG of a result set
    result_df: DataFrame. The dataframe containing the results
    """
    result_df = ranked_list.copy()
    result_df['docno'] = result_df['docno'].astype(str)
    # pair = (qid, result_df['docid'].values[0])
    
    result_df['gt_relevance'] = result_df.apply(lambda row: qrel_dict.get((qid, str(row['docno'])), 0), axis=1)
    scores = result_df['gt_relevance'].values[0:n]
    ideal_scores = sorted(scores, reverse=True)
    DCG = dcg(scores, n)
    IDCG = dcg(ideal_scores, n)
    if IDCG >0:
        return DCG/IDCG
    else:
        return 0
    
def disc(k):
    """
    The discount function used in the nDCG/AWRF
    """
    return 1 / np.log2(max(k, 2))

def calculate_awrf(ranked_list, expected_distribution, attribute, n=10):
    """
    Calculate the AWRF of a result set
    result_df: DataFrame. The dataframe containing the results
    expected_distribution: Dict. The distribution of the dataset. e.g.
            {'Man': 1494634,
            'Woman': 353495,
            'Non-binary': 781,
            'Unknown': 4605703,
            'null': 0}
    attribute: Str. The attribute to consider. e.g. 'gender'
    n: Int.  refer to the number of documents to consider
    """
    result_df = ranked_list.copy()
    current_distribution = {key: 0 for key in expected_distribution}
    for index, row in result_df.iterrows():
        if index<n:
            if row[attribute]:
                current_distribution[row[attribute]]+= disc(index+1)
            else:
                current_distribution['null']+= disc(index+1)
    
    desired_distribution = np.array(list(expected_distribution.values()))
    current_distribution = np.array(list(current_distribution.values()))

    target_distr = desired_distribution/np.sum(desired_distribution)
    current_distr = current_distribution/np.sum(current_distribution)
    awrf = 1- jensenshannon(target_distr, current_distr)**2
    return awrf

# 2. Obtain dataset distribution of gender and qrel for AWRF and NDCG calculation

In [6]:
dataset = pt.get_dataset('irds:trec-fair/2022/train')
dataset_distribution = {'Man':0, 'Woman':0, 'Non-binary':0, 'Unknown':0, 'null':0}
for it in dataset.get_corpus_iter():
    title = it['title']
    if it['gender_category']:
        gender = it['gender_category']
        dataset_distribution[gender]+=1


dataset = ir_datasets.load("trec-fair/2022/train")
qrel_dict = {(qrel.query_id, qrel.doc_id): qrel.relevance for qrel in dataset.qrels_iter()}

trec-fair/2022/train documents:   0%|          | 0/6475537 [00:00<?, ?it/s]

[INFO] If you have a local copy of https://data.boisestate.edu/library/Ekstrand/TRECFairRanking/2022/trec_2022_train_reldocs.jsonl, you can symlink it here to avoid downloading it again: C:\Users\WalterKahn\.ir_datasets\downloads\d132b4cc8c6c75525479728321db5176
[INFO] [starting] https://data.boisestate.edu/library/Ekstrand/TRECFairRanking/2022/trec_2022_train_reldocs.jsonl
[INFO] [finished] https://data.boisestate.edu/library/Ekstrand/TRECFairRanking/2022/trec_2022_train_reldocs.jsonl: [00:06] [18.0MB] [2.64MB/s]


# 3. Obtain biography related queries

In [8]:
biography_related_topics = ['396', '397', '403', '770', '1371']
with open("train_topics_meta.jsonl", "r") as f:
    topics = [json.loads(line) for line in f]
# "id":396,"title":"Biography/WikiProject Actors and Filmmakers"
# "id":397,"title":"Biography/WikiProject Musicians"
# "id":403,"title":"Biography/science and academia work group"
# "id":770,"title":"Crime and Criminal Biography"
# "id":1371,"title":"Japan/Biography task force"
topic_id = topics[0]['id']
all_topic_queries = topics[0]['keywords']
test_query_dict = {}
for key, val in topic_id.items():
    if str(val) in biography_related_topics:
        topic_queries = all_topic_queries[key]
        topic_query_list = ast.literal_eval(topic_queries)
        test_query_dict[str(val)] = topic_query_list


# 4. Build the retrieval pipeline

In [11]:
index_ref = pt.IndexRef.of('C:/Users/WalterKahn/Documents/F - Overig/InfoRetrieval/Git/ir-group-30/trec-fair_2022_multi')
pipeline = pt.BatchRetrieve(index_ref, wmodel='BM25', metadata=['gender', 'gender_category', 'title', 'docno'])

09:17:47.159 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
09:17:48.597 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 7,3 GiB of memory would be required.


# 5. Retrieve and evaluation

In [40]:
eval_reulst = []
from tqdm import tqdm
for topic_id, topic_query_list in tqdm(test_query_dict.items()):
    for query in topic_query_list:
        result = pipeline.search(query)
        
        nDCG5 = calculate_ndcg(result, qrel_dict, qid=topic_id, n=5)
        awrf5 = calculate_awrf(result, dataset_distribution, 'gender_category', n=5)
        
        nDCG10 = calculate_ndcg(result, qrel_dict, qid=topic_id, n=10)
        awrf10 = calculate_awrf(result, dataset_distribution, 'gender_category', n=10)
        
        nDCG20 = calculate_ndcg(result, qrel_dict, qid=topic_id, n=20)
        awrf20 = calculate_awrf(result, dataset_distribution, 'gender_category', n=20)
        
        nDCG50 = calculate_ndcg(result, qrel_dict, qid=topic_id, n=50)
        awrf50 = calculate_awrf(result, dataset_distribution, 'gender_category', n=50)
        
        # eval_reulst.append([topic_id, query, nDCG5, awrf5, nDCG10, awrf10, nDCG20, awrf20, nDCG50, awrf50])
        eval_reulst.append([topic_id, query, nDCG5,nDCG10,nDCG20,nDCG50, awrf5,awrf10,awrf20,awrf50])

eval_df = pd.DataFrame(eval_reulst, columns=["topic_id", "query", "nDCG5","nDCG10","nDCG20","nDCG50", "awrf5","awrf10","awrf20","awrf50"])

100%|██████████| 5/5 [00:50<00:00, 10.17s/it]


In [94]:
eval_df.to_csv("BM25 results.csv",index=False)